In [6]:
file_path = 'data/train.txt'

with open(file_path, 'r') as f:
    first_line = f.readline()
    print(first_line.strip())

The boy went to a video arcade. He played his favorite machine. His games didn't go very well. He told the owner about his experience. The owner explained that he had made the game settings harder.
